In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
#set tile to predict
tile_to_predict = 't15'

In [ ]:
    
def Gen_Test_Train(chrom):
    #set tile to be used as test set
    os.system("grep '_chr%s_' '/u/home/m/mudiyang/scratch/Scaleup_counts_sequences/DNACOUNTS/cords.txt' > '/u/home/m/mudiyang/scratch/Scaleup_counts_sequences/DNACOUNTS/chr%s.txt'"
             %(chrom,chrom))
    
    
    #use cell_prom_rep filed to pick which cell/prom/rep data to process, and later use for the regressor.
    #options are: ['Hep_Sv40_1','Hep_Sv40_2','Hep_min_1','Hep_min_2','K_Sv40_1','K_Sv40_2','K_min_1','K_min_2']
    Data_path = "/u/home/m/mudiyang/scratch/MPRAnalyze/"
    rna_path       = "%s/RNA_df.csv"%Data_path
    dna_path       = "%s/DNA_df.csv"%Data_path
    cell_prom_rep  ="Hep_Sv40_1" 

    rna = pd.read_csv(rna_path, header = 0, index_col = 0, sep = '\t')
    dna = pd.read_csv(dna_path, header = 0, index_col = 0, sep = '\t')

    missing = pd.read_pickle('/u/home/m/mudiyang/scratch/Scaleup_counts_sequences/DNACOUNTS/missing.pkl')

    rna = pd.concat([rna, missing])
    dna = pd.concat([dna, missing])

    dna_values = np.array(dna['%s'%cell_prom_rep])
    rna_values = np.array(rna['%s'%cell_prom_rep])

    dna_matrix = dna_values.reshape(15720,31)
    rna_matrix = rna_values.reshape(15720,31)


    indexes = np.genfromtxt('/u/home/m/mudiyang/scratch/Scaleup_counts_sequences/DNACOUNTS/cords.txt', 
                            dtype=str, delimiter ='\n')

    tiles = ['t%s']*31
    numbers = list(range(1,32,1))

    for i in range(0,len(tiles),1):
        tiles[i] = tiles[i]%numbers[i]

    rna = pd.DataFrame(rna_matrix, columns = tiles, index = indexes)
    dna = pd.DataFrame(dna_matrix, columns = tiles, index = indexes)



    #normalize data
    #add pesudo count to smooth data
    rna = rna+1
    dna = dna+1


    dna_no_nan = dna.fillna(0)
    rna_no_nan = rna.fillna(0)


    #get sums of DNA and RNA counts
    dna_sum = dna_no_nan.values.sum()
    rna_sum = rna_no_nan.values.sum()


    #generate normalized counts. I am including the strange multiply dna_sum by two step, 
    #but we can comment that out if we diecide that does not make sense. 
    rna_counts = rna/rna_sum

    dna_sum = dna_sum*2
    dna_counts = dna/dna_sum


    #generate scores for each tile. These will be used as the labels for the regressor
    tile_scores = np.log2(rna_counts/dna_counts)

    #remove regions that are in chrom and reserve as test set
    remove = index = np.genfromtxt('/u/home/m/mudiyang/scratch/Scaleup_counts_sequences/DNACOUNTS/chr%s.txt'%chrom, 
                            dtype=str, delimiter ='\n')
    
    #drop all NaN values
    tile_scores = tile_scores.dropna()

    test_set = tile_scores[tile_scores.index.isin(remove)]
    train_set = tile_scores[~tile_scores.index.isin(remove)]
    del tile_scores

    y_train = pd.DataFrame(train_set['%s'%tile_to_predict])
    y_test = pd.DataFrame(test_set['%s'%tile_to_predict])

    X_train = train_set.drop(columns = ['%s'%tile_to_predict])
    X_test = test_set.drop(columns = ['%s'%tile_to_predict])
    
    X_train.to_pickle('/u/home/m/mudiyang/scratch/Tile_Prediction_Regressors/Data/CV_t15_rotate_chrs/X_train_chr%s.pkl'%chrom)
    X_test.to_pickle('/u/home/m/mudiyang/scratch/Tile_Prediction_Regressors/Data/CV_t15_rotate_chrs/X_test_chr%s.pkl'%chrom)
    y_train.to_pickle('/u/home/m/mudiyang/scratch/Tile_Prediction_Regressors/Data/CV_t15_rotate_chrs/y_train_chr%s.pkl'%chrom)
    y_test.to_pickle('/u/home/m/mudiyang/scratch/Tile_Prediction_Regressors/Data/CV_t15_rotate_chrs/y_test_chr%s.pkl'%chrom)

In [ ]:
for i in range(1,23,1):
    Gen_Test_Train(i)

In [ ]:
Gen_Test_Train('X')